In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import torch
import torchtext
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from imblearn.over_sampling import SMOTE
import random
import time
from random import randint
from googletrans import Translator
import re
import openai

In [ ]:
openai.api_key = 'chave'

In [2]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

LEARNING_RATE = 0.005
BATCH_SIZE = 100
NUM_EPOCHS = 20
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 100
HIDDEN_DIM = 256
NUM_CLASSES = 2

In [3]:
df = pd.read_csv('spam6.csv')
df

,Mensagem,Classe
0,Anything lor if they all go then i go lor...,ham
1,For taking part in our mobile survey yesterday...,spam
2,65-321\n\nPRIVATE! Your 2003 Account Statement...,spam
3,"Bears Pic Nick, and Tom, Pete and ... Dick. In...",spam
4,Does uncle timi help in clearing cars,ham
...,...,...
459,"URGENT! Your mobile was awarded a å£1,500 Bonu...",spam
460,"Sorry, I'll call you later. I am in meeting sir.",ham
461,Get the Nokia tone for your mobile phone absol...,spam
462,"I'm having a bite to eat now, but I'll be head...",ham


In [ ]:
# paraprhasing with pre-trained model gpt-3

j = 0
new = []
sentence = "Rewrite a phrase preserving their original meaning: " 

for i in range (230,232):
    var = df[df.index == j]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=sentence+var,
        temperature=0,
        max_tokens=500,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    aux = response.choices[0].text[2:]
    new.append(aux)
    j += 1

In [ ]:
# Random Deletion

j = 0
new = []

for i in range(0,232):
    var = df[df.index == j]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) <= 5:
        rand = randint(0,len(spl)-1)
        spl.pop(rand)
        aux = ' '.join(spl)
        new.append(aux)
    elif len(spl) <= 20:
        for n in range(0,5):
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
        aux = ' '.join(spl)
        new.append(aux)
    elif len(spl) <= 50:
        for n in range(0,15):
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
        aux = ' '.join(spl)
        new.append(aux)
    else:
        for n in range(0,40):
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
        aux = ' '.join(spl)
        new.append(aux)
    j += 1

In [ ]:
# back translation

translator = Translator()

j = 0
new = []

for i in range(0,232):
    var = df[df.index == j]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    trans1 = translator.translate(var, dest='pt', src='en').text
    trans2 = translator.translate(trans1, dest='en', src='pt').text
    new.append(trans2)
    j += 1

In [ ]:
# Random Swap

j = 0
new = []

for i in range(0,232):
    var = df[df.index == j]
    var = var['Mensagem'].to_string()
    var = var[1:].strip()
    spl = var.split()
    for n in range(0,2):
        rand1 = randint(0,len(spl)-1)
        rand2 = randint(0,len(spl)-1)
        aux1 = spl.pop(rand1)
        spl.insert(rand2, aux1)
    aux2 = ' '.join(spl)
    new.append(aux2)
    j += 1

In [ ]:
novo = pd.DataFrame(new)
novo['Classe'] = df['Classe']
novo.rename(columns={0: 'Mensagem'}, inplace = True)
novo

In [ ]:
novo = novo.drop(232)


In [ ]:
novo

In [ ]:
fim = df.append(novo, ignore_index=True)

In [ ]:
del(df)

In [ ]:
del(novo)

In [ ]:
fim = fim.sample(frac = 1)

In [ ]:
fim = fim.reset_index(drop=True)
fim

In [ ]:
fim.to_csv('spam6.csv', index=None)

In [ ]:
#out = df.sample(n = 1250)
#out

In [ ]:
#df = df.drop(out.index)

In [ ]:
#df = df.drop(out.index)
#df

In [ ]:
#df = df.reset_index(drop=True)
#df

In [ ]:
#df.to_csv('spam3.csv', index=None)

In [ ]:
#df = df[['Mensagem', 'Classe']]

In [ ]:
#df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [4]:
### Defining the feature processing

TEXT = torchtext.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.data.LabelField(dtype=torch.long)

In [5]:
fields = [('Mensagem', TEXT), ('Classe', LABEL)]

dataset = torchtext.data.TabularDataset(
    path='spam6.csv', format='csv',
    skip_header=True, fields=fields)

In [6]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 371
Num Test: 93


In [7]:
print(vars(train_data.examples[0]))

{'Mensagem': ['Purchase', 'a', 'pizza', ',', 'either', 'Meat', 'Lovers', 'or', 'Supreme', '.'], 'Classe': 'ham'}


In [8]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 2095
Number of classes: 2


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 319), ('to', 174), (',', 169), ('!', 151), ('a', 108), ('you', 106), ('I', 91), ('?', 78), ('...', 70), ('for', 69), ('the', 68), ('is', 64), ('your', 56), ('have', 53), ('and', 48), ('are', 41), ('2', 39), ('/', 39), ("'s", 38), ('on', 38)]


In [10]:
print(TEXT.vocab.itos[:10]) # itos = integer-to-string

['<unk>', '<pad>', '.', 'to', ',', '!', 'a', 'you', 'I', '?']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'ham': 0, 'spam': 1})


In [12]:
LABEL.vocab.freqs

Counter({'ham': 193, 'spam': 178})

In [13]:
train_loader ,test_loader = \
        torchtext.data.BucketIterator.splits(
        (train_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.Mensagem),
         device=DEVICE
    )

In [ ]:
for i in enumerate(train_loader):
    print(i)

In [ ]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.Mensagem.size()}')
    print(f'Target vector size: {batch.Classe.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.Mensagem.size()}')
    print(f'Target vector size: {batch.Classe.size()}')
    break

In [14]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

In [15]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

RNN(
  (embedding): Embedding(2095, 100)
  (rnn): LSTM(100, 256)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


In [16]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
    return correct_pred.float()/num_examples * 100

In [19]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.Mensagem.to(DEVICE)
        labels = batch_data.Classe.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: 'f'{compute_accuracy(model, train_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/020 | Batch 000/004 | Loss: 0.4373
Epoch: 001/020 | Batch 001/004 | Loss: 0.4643
Epoch: 001/020 | Batch 002/004 | Loss: 0.3099
Epoch: 001/020 | Batch 003/004 | Loss: 0.3340
training accuracy: 94.34%
Time elapsed: 0.02 min
Epoch: 002/020 | Batch 000/004 | Loss: 0.2984
Epoch: 002/020 | Batch 001/004 | Loss: 0.3278
Epoch: 002/020 | Batch 002/004 | Loss: 0.3217
Epoch: 002/020 | Batch 003/004 | Loss: 0.3304
training accuracy: 94.07%
Time elapsed: 0.04 min
Epoch: 003/020 | Batch 000/004 | Loss: 0.2357
Epoch: 003/020 | Batch 001/004 | Loss: 0.1984
Epoch: 003/020 | Batch 002/004 | Loss: 0.2137
Epoch: 003/020 | Batch 003/004 | Loss: 0.2195
training accuracy: 96.23%
Time elapsed: 0.06 min
Epoch: 004/020 | Batch 000/004 | Loss: 0.1454
Epoch: 004/020 | Batch 001/004 | Loss: 0.1224
Epoch: 004/020 | Batch 002/004 | Loss: 0.1181
Epoch: 004/020 | Batch 003/004 | Loss: 0.1556
training accuracy: 99.19%
Time elapsed: 0.07 min
Epoch: 005/020 | Batch 000/004 | Loss: 0.0813
Epoch: 005/020 | Batch

In [ ]:
import spacy


nlp = spacy.blank("en")

def predict(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    prediction = torch.max(prediction, 1)
    return prediction

In [ ]:
predict(model, "do you want a gift, click here")